In [ ]:
using ProgressMeter, PyCall, PyPlot, Images, HDF5,NaNStatistics, Statistics, DSP, Lasso, JLD2
using _Data, _Math

In [ ]:
@pyimport numpy
@pyimport sklearn.decomposition as decomposition
@pyimport scipy.stats as stats
@pyimport matplotlib.colors as mpl_colors
@pyimport matplotlib.cm as cm 
@pyimport sklearn.cluster as cluster
@pyimport seaborn

@pyimport matplotlib.pyplot as py_plt

In [ ]:
rc_params = PyDict(pyimport("matplotlib")["rcParams"]);
rc_params["font.sans-serif"] = ["Arial"];
rc_params["font.size"] = 7;
rc_params["lines.linewidth"] = 1;
rc_params["lines.markersize"] = 4;
rc_params["xtick.major.size"] = 2;
rc_params["ytick.major.size"] = 2;
rc_params["xtick.major.pad"] = 2;
rc_params["ytick.major.pad"] = 2;
# rc_params["axes.labelpad"] = 2;
rc_params["axes.spines.right"] = false
rc_params["axes.spines.top"] = false

cim(img::Matrix{UInt32}) = CairoImageSurface(img, Cairo.FORMAT_RGB24; flipxy = false) 
cim(img::Matrix{ARGB32}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{RGB24}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{UInt8}) = cim(Gray.(reinterpret(N0f8, img)))
cim(img::Array{UInt8,3}) = cim(RGB24.(reinterpret(N0f8, img[:,:,1]), reinterpret(N0f8, img[:,:,2]), reinterpret(N0f8, img[:,:,3])));downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);
downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);

In [ ]:
fig_path = "/home/chuyu/Notebooks/project_place_cell/figures/output/review_figures/ABA"
mkpath(fig_path)

In [ ]:
include("../../../functions/func_map.jl")
include("../../../functions/func_stat.jl")
include("../../../functions/func_data.jl")
include("../../../functions/func_plot.jl")

In [ ]:
data_info_all = [
    ["20240128_132056", 1, "20240128_152133", 1, "20240128_164754", 1, "drew"],
    ["20240127_185226", 8, "20240127_202210", 8, "20240127_212910", 8, "jen"],
    ["20240127_120909_20240127_132316", 1, "20240127_152329", 1, "20240127_164645", 1, "chuyu"],
    ["20240128_174842_20240128_182823", 8, "20240128_194250", 8, "20240128_203533", 8, "chuyu"],

    ]

In [ ]:
analyzer = "chuyu"

In [ ]:
PF_AA = []
PV_AA = []
PS_AA = []

PF_AB = []
PV_AB = []
PS_AB = []

PF_BA = []
PV_BA = []
PS_BA = []

for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]

    experiment_filename_2 = data_info[3]
    server_2 = data_info[4]
    
    experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_2 = Dataset(experiment_filename_2, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")


    
    
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_matchangle.h5")
    file = h5open(info_filename, "r")
    confined_place_cell_index_AB = HDF5.readmmap(file["confined_place_cell_index"])
    tel_place_cell_index_AB = HDF5.readmmap(file["tel_place_cell_index"])
    corr_trials_all_AB = HDF5.readmmap(file["corr_trials_all"])
    pv_map_normalized_AB = HDF5.readmmap(file["pv_map_dFF"])
    peak_shift_mm_AB = HDF5.readmmap(file["peak_shift_mm"])
    close(file)

        
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_AA.h5")
    file = h5open(info_filename, "r")
    tel_place_cell_index_AA = HDF5.readmmap(file["tel_place_cell_index"])
    confined_place_cell_index_AA = HDF5.readmmap(file["confined_place_cell_index"])
    corr_trials_all_AA = HDF5.readmmap(file["corr_trials_all"])
    pv_map_normalized_AA = HDF5.readmmap(file["pv_map_dFF"])
    peak_shift_mm_AA = HDF5.readmmap(file["peak_shift_mm"])
    close(file)


    info_filename = joinpath(data_path(ds_save_cy_2), "compare_map_results_BA_matchangle.h5")
    file = h5open(info_filename, "r")
    tel_place_cell_index_BA = HDF5.readmmap(file["tel_place_cell_index"])
    confined_place_cell_index_BA = HDF5.readmmap(file["confined_place_cell_index"])
    corr_trials_all_BA = HDF5.readmmap(file["corr_trials_all"])
    pv_map_normalized_BA = HDF5.readmmap(file["pv_map_dFF"])
    peak_shift_mm_BA = HDF5.readmmap(file["peak_shift_mm"])
    close(file)


  
        
    append!(PF_AA, [corr_trials_all_AA[tel_place_cell_index_AA]])
    append!(PV_AA, [pv_map_normalized_AA])
    append!(PS_AA, [peak_shift_mm_AA])

    append!(PF_AB, [corr_trials_all_AB[tel_place_cell_index_AB]])
    append!(PV_AB, [pv_map_normalized_AB])
    append!(PS_AB, [peak_shift_mm_AB])

    append!(PF_BA, [corr_trials_all_BA[tel_place_cell_index_BA]])
    append!(PV_BA, [pv_map_normalized_BA])
    append!(PS_BA, [peak_shift_mm_BA])


    println(length(union(tel_place_cell_index_AA, tel_place_cell_index_AB, tel_place_cell_index_BA)))
    
    
end




In [ ]:
@pyimport statannotations.Annotator as Annotator
@pyimport seaborn as sns
@pyimport pandas as pd

In [ ]:
function fill_nan(vec, desired_length)
    original_length = length(vec)
    new_vec = fill!(Array{Float64,1}(undef, desired_length), NaN)
    new_vec[1:original_length] = vec
    return new_vec
end

In [ ]:
compare_data = Dict()
compare_data["A1-B"] = reduce(vcat, PS_AB)
compare_data["B-A2"] = reduce(vcat, PS_BA)
compare_data["A1-A2"]= reduce(vcat, PS_AA);

In [ ]:
desired_length = maximum(length(compare_data[key]) for key in keys(compare_data))

for key in keys(compare_data)
    compare_data[key] = fill_nan(compare_data[key], desired_length)
end
df = pd.DataFrame(compare_data)

In [ ]:
results_all = [PF_AA, PF_AB, PF_BA]


for which_fish = 1:length(results_all[1])

formatted_pvalues = []


compare_pairs = [("A1-B", "A1-A2"),("A1-B", "B-A2")]   

for (i,j) in [(2,1), (2,3)]
    x = reduce(vcat, results_all[i][which_fish])
    y = reduce(vcat, results_all[j][which_fish])
    x = x[isfinite.(x)];
    y = y[isfinite.(y)];
    _, p_value = stats.mannwhitneyu(x, y, alternative ="less")
    
    println(p_value)
    siginficance = "n.s."
    if p_value < 0.01
        siginficance = "*"
    end
    if p_value < 1e-3
        siginficance = "**"
    end
    if p_value < 1e-5
        siginficance = "***"
    end
    append!(formatted_pvalues, [siginficance])
end

fig, ax = subplots(1,1,figsize=(0.8, 0.8))


dataset = []
for distribution in results_all
    plot_distribution= distribution[which_fish]
    plot_distribution = plot_distribution[isfinite.(plot_distribution)];
    append!(dataset, [plot_distribution])
end

parts = py_plt.violinplot(dataset=dataset,positions=[0,1,2], showmeans=false, showextrema=false, showmedians=true)

ylim(-1,1)

for (i, pc) in enumerate(parts["bodies"])
    pc.set_facecolor("gray")
end

parts["cmedians"].set_edgecolor("k")

    
# sns.violinplot(order = ["A1-B","B-A2", "A1-A2"], data=df,ax=ax)
xticks([0,1,2], ["A1-A2", "A1-B", "B-A2"], rotation=45)

    
if which_fish == 1
    ylabel("PF cor.", labelpad = 0)
else
    yticks([],[])
end


annotator = Annotator.Annotator(ax, compare_pairs, data=df, order=["A1-A2", "A1-B","B-A2"])
annotator.configure(loc="outside")
annotator.configure(test="Mann-Whitney", text_format="simple", loc="outside",show_test_name=false)
annotator.set_custom_annotations(formatted_pvalues)
annotator.annotate()
# annotator.apply_and_annotate()


fig.savefig(joinpath(fig_path, "PF_correlation_fish$(which_fish).pdf"), bbox_inches="tight",transparent = true);

end

In [ ]:
results_all = [PV_AA, PV_AB, PV_BA]


for which_fish = 1:length(results_all[1])

formatted_pvalues = []


compare_pairs = [("A1-B", "A1-A2"),("A1-B", "B-A2")]   

for (i,j) in [(2,1), (2,3)]
    x = reduce(vcat, results_all[i][which_fish])
    y = reduce(vcat, results_all[j][which_fish])
    x = x[isfinite.(x)];
    y = y[isfinite.(y)];
    _, p_value = stats.mannwhitneyu(x, y,  alternative="less")
    println(p_value)
    
    
    siginficance = "n.s."
    if p_value < 0.01
        siginficance = "*"
    end
    if p_value < 1e-3
        siginficance = "**"
    end
    if p_value < 1e-5
        siginficance = "***"
    end
    append!(formatted_pvalues, [siginficance])
end

fig, ax = subplots(1,1,figsize=(0.8, 0.8))


dataset = []
for distribution in results_all
    plot_distribution= distribution[which_fish]
    plot_distribution = plot_distribution[isfinite.(plot_distribution)];
    append!(dataset, [plot_distribution])
end

parts = py_plt.violinplot(dataset=dataset,positions=[0,1,2], showmeans=false, showextrema=false, showmedians=true)

ylim(-0.5,0.7)

for (i, pc) in enumerate(parts["bodies"])
    pc.set_facecolor("gray")
end

parts["cmedians"].set_edgecolor("k")

    
# sns.violinplot(order = ["A1-B","B-A2", "A1-A2"], data=df,ax=ax)
xticks([0,1,2], ["A1-A2", "A1-B", "B-A2"], rotation=45)
if which_fish == 1
    ylabel("PV cor.", labelpad = 0)
else
    yticks([],[])
end


annotator = Annotator.Annotator(ax, compare_pairs, data=df, order=["A1-A2", "A1-B","B-A2"])
annotator.configure(loc="outside")
annotator.configure(test="Mann-Whitney", text_format="simple", loc="outside",show_test_name=false)
annotator.set_custom_annotations(formatted_pvalues)
annotator.annotate()
# annotator.apply_and_annotate()


    
fig.savefig(joinpath(fig_path, "PV_correlation_fish$(which_fish).pdf"), bbox_inches="tight",transparent = true);

end

In [ ]:
results_all = [PS_AA, PS_AB, PS_BA]


for which_fish = 1:length(results_all[1])

formatted_pvalues = []


compare_pairs = [("A1-B", "A1-A2"),("A1-B", "B-A2")]   

for (i,j) in [(2,1), (2,3)]
    x = reduce(vcat, results_all[i][which_fish])
    y = reduce(vcat, results_all[j][which_fish])
    x = x[isfinite.(x)];
    y = y[isfinite.(y)];
    _, p_value = stats.mannwhitneyu(x, y, alternative="greater")
    println(p_value)
    
    
    siginficance = "n.s."
    if p_value < 0.01
        siginficance = "*"
    end
    if p_value < 1e-3
        siginficance = "**"
    end
    if p_value < 1e-5
        siginficance = "***"
    end
    append!(formatted_pvalues, [siginficance])
end

fig, ax = subplots(1,1,figsize=(0.8, 0.8))


dataset = []
for distribution in results_all
    plot_distribution= distribution[which_fish]
    plot_distribution = plot_distribution[isfinite.(plot_distribution)];
    append!(dataset, [plot_distribution])
end

parts = py_plt.violinplot(dataset=dataset,positions=[0,1,2], showmeans=false, showextrema=false, showmedians=true)



if which_fish == 3
    scatter(numpy.repeat(2, length(results_all[3][which_fish])), results_all[3][which_fish], s= 3, color="gray", alpha=0.5)
end

ylim(-0.5,0.7)

for (i, pc) in enumerate(parts["bodies"])
    pc.set_facecolor("gray")
end

parts["cmedians"].set_edgecolor("k")

    
# sns.violinplot(order = ["A1-B","B-A2", "A1-A2"], data=df,ax=ax)
xticks([0,1,2], ["A1-A2", "A1-B", "B-A2"], rotation=45)
if which_fish == 1
    ylabel("PF shift (mm)", labelpad = 0)
else
    yticks([],[])
end
ylim(-3,50)

annotator = Annotator.Annotator(ax, compare_pairs, data=df, order=["A1-A2", "A1-B","B-A2"])
annotator.configure(loc="outside")
annotator.configure(test="Mann-Whitney", text_format="simple", loc="outside",show_test_name=false)
annotator.set_custom_annotations(formatted_pvalues)
annotator.annotate()
# annotator.apply_and_annotate()
fig.savefig(joinpath(fig_path, "peak_shift_fish$(which_fish).pdf"), bbox_inches="tight",transparent = true);

end

In [ ]:
fig_path

In [ ]:
data_dict = Dict()

which_fish = 1
results_all = [PF_AA, PF_AB, PF_BA, PV_AA, PV_AB, PV_BA, PS_AA, PS_AB, PS_BA]


compare_pairs = ["PF cor., A1-B", "PF cor., A1-A2", "PF cor., B-A2", "PV cor., A1-B", "PV cor., A1-A2", "PV cor., B-A2", "PF shift, A1-B", "PF shift, A1-A2", "PF shift, B-A2"]   
for i in 1:length(compare_pairs)
    sample_size = []
    for which_fish in 1:length(PF_AA)
        x = reduce(vcat, results_all[i][which_fish])
        x = x[isfinite.(x)];
        append!(sample_size, length(x))
    end
    data_dict[compare_pairs[i]] = sample_size
end


In [ ]:
df = pd.DataFrame.from_dict(data_dict)
df = df.sort_index(axis=1)

In [ ]:
@pyimport os
@pyimport openpyxl

save_path = "../../../figures/source_data/Fig. 9e sample size.xlsx"
sheet_name = "Fig. 9e"

exists = os.path.isfile(save_path)
if exists
    writer = pd.ExcelWriter(save_path, engine = "openpyxl", mode="a")
    df.to_excel(writer, sheet_name = sheet_name)
    writer.close()
else
    df.to_excel(save_path, sheet_name)  
end